# Project - Formula 1 Analysis
## Race Status + Drivers and Teams


  ***

In [3]:
# Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import FileLink

In [4]:
# Kaggle API Conection

import kagglehub

# Download latest version
path = kagglehub.dataset_download("rohanrao/formula-1-world-championship-1950-2020")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Joaquim Meruje\.cache\kagglehub\datasets\rohanrao\formula-1-world-championship-1950-2020\versions\24


In [5]:
# Read Datasets

# Paths for CSV files
races_file = f"{path}/races.csv"
results_file = f"{path}/results.csv"
circuits_file = f"{path}/circuits.csv"
status_file = f"{path}/status.csv"
driver_standings_file = f"{path}/driver_standings.csv"
lap_times_file = f"{path}/lap_times.csv"
drivers_file = f"{path}/drivers.csv"
constructors_file = f"{path}/constructors.csv"
constructors_standings_file = f"{path}/constructor_standings.csv"

# Read for a dataframe each csv file
races = pd.read_csv(races_file, na_values='\\N')
results = pd.read_csv(results_file, na_values='\\N')
circuits = pd.read_csv(circuits_file, na_values='\\N')
status = pd.read_csv(status_file, na_values='\\N')
driver_standings = pd.read_csv(driver_standings_file, na_values='\\N')
lap_times = pd.read_csv(lap_times_file, na_values='\\N')
drivers = pd.read_csv(drivers_file, na_values='\\N')
constructors = pd.read_csv(constructors_file, na_values='\\N')
constructors_standings = pd.read_csv(constructors_standings_file, na_values='\\N')

## 1) Race Status

### 1.1) Status Results for Driver

In [96]:
# Create a new dataframe for save race status for each driver

results_by_status = results.groupby(['driverId', 'statusId']).size()
results_by_status = results_by_status.reset_index(name='count')
results_by_status.head(10)

,driverId,statusId,count
0,1,1,312
1,1,2,2
2,1,3,1
3,1,4,12
4,1,5,3
5,1,6,3
6,1,9,1
7,1,10,1
8,1,11,10
9,1,20,2


In [97]:
# Merge status problems name with status id 

results_by_status_total = results_by_status.merge(status[['statusId', 'status']], on='statusId', how='right')
results_by_status_total = results_by_status_total.merge(drivers[['driverId', 'forename', 'surname']], on='driverId', how='right')

# Create a new column to have the complete names

results_by_status_total['Name'] = results_by_status_total['forename'] + ' ' + results_by_status_total['surname']
results_by_status_total = results_by_status_total.drop(columns=['forename', 'surname'])

results_by_status_total.head(10)

,driverId,statusId,count,status,Name
0,1.0,1,312.0,Finished,Lewis Hamilton
1,1.0,2,2.0,Disqualified,Lewis Hamilton
2,1.0,3,1.0,Accident,Lewis Hamilton
3,1.0,4,12.0,Collision,Lewis Hamilton
4,1.0,5,3.0,Engine,Lewis Hamilton
5,1.0,6,3.0,Gearbox,Lewis Hamilton
6,1.0,9,1.0,Hydraulics,Lewis Hamilton
7,1.0,10,1.0,Electrical,Lewis Hamilton
8,1.0,11,10.0,+1 Lap,Lewis Hamilton
9,1.0,20,2.0,Spun off,Lewis Hamilton


In [98]:
# Filter dataset to count total races by each driver 
df_filtered = results_by_status_total[results_by_status_total['statusId'] != 1]
race_counts = results_by_status_total.groupby('Name')['count'].sum().reset_index(name='Total Races')

# Compute the number of dnfs by each driver and merge dataframes
dnfs = df_filtered.groupby('Name').size().reset_index(name='Total DNFs')
race_counts = pd.merge(race_counts, dnfs, on='Name', how='inner')

# Compute a new column with the Finished Races in percentage (%)
race_counts['Finished Races (%)'] = ((1 - (race_counts['Total DNFs'] / race_counts['Total Races']))*100).round(2)

# Sort and filter dataset by total races
race_counts = race_counts[race_counts['Total Races'] > 30]
race_counts.sort_values(by='Total DNFs', ascending=False)

race_counts.head(10)

,Name,Total Races,Total DNFs,Finished Races (%)
2,Adrian Sutil,128.0,17,86.72
4,Aguri Suzuki,88.0,20,77.27
8,Alain Prost,202.0,27,86.63
11,Alan Jones,117.0,27,76.92
16,Alberto Ascari,36.0,9,75.00
22,Alessandro Nannini,77.0,25,67.53
24,Alessandro Zanardi,44.0,18,59.09
27,Alex Caffi,75.0,20,73.33
31,Alexander Albon,105.0,13,87.62
33,Alexander Wurz,69.0,10,85.51


In [99]:
# Merge results and status datasets
results_with_status = results.merge(status, on='statusId')
results_with_status = results_with_status[['driverId', 'status']]

# Merge datasets to get drivers names
results_with_status = results_with_status.merge(drivers[['driverId', 'forename', 'surname']], on='driverId', how='right')

# Create a new column to have the complete names
results_with_status['Name'] = results_with_status['forename'] + ' ' + results_with_status['surname']
results_with_status = results_with_status.drop(columns=['forename', 'surname'])

# Create a nee dataframe with all status problems columns
results_with_status = pd.crosstab(results_with_status['Name'], results_with_status['status'])

results_with_status.head(10)

status,+1 Lap,+10 Laps,+11 Laps,+12 Laps,+13 Laps,+14 Laps,+15 Laps,+16 Laps,+17 Laps,+18 Laps,...,Vibrations,Water leak,Water pipe,Water pressure,Water pump,Wheel,Wheel bearing,Wheel nut,Wheel rim,Withdrew
Name,,,,,,,,,,,,,,,,,,,,,
Adolf Brudes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Adolfo Cruz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Adrian Sutil,38,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
Adrián Campos,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Aguri Suzuki,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Al Herman,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Al Keller,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Al Pease,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Alain Prost,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [100]:
# Get all columns related to + Laps together
columns_with_lap = [col for col in results_with_status.columns if 'Lap' in col]

# Filter the dataframe to have just that columns
status_laps = results_with_status[columns_with_lap]

# Sum all columns about + Laps to count how much laps they had in their careers
status_laps['Total'] = status_laps[columns_with_lap].sum(axis=1)

# Drop all the previous + Laps columns
status_laps = status_laps.drop(columns=columns_with_lap)

C:\Users\Joaquim Meruje\AppData\Local\Temp\ipykernel_36732\1420740539.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  status_laps['Total'] = status_laps[columns_with_lap].sum(axis=1)


In [101]:
# Merge the total laps in the race_counts dataframe
race_counts = race_counts.merge(status_laps, on='Name', how='left')

# Rename the column name
race_counts.rename(columns={'Total': 'Total + Laps'}, inplace=True)

race_counts.head(10)

,Name,Total Races,Total DNFs,Finished Races (%),Total + Laps
0,Adrian Sutil,128.0,17,86.72,49
1,Aguri Suzuki,88.0,20,77.27,22
2,Alain Prost,202.0,27,86.63,18
3,Alan Jones,117.0,27,76.92,29
4,Alberto Ascari,36.0,9,75.00,5
5,Alessandro Nannini,77.0,25,67.53,12
6,Alessandro Zanardi,44.0,18,59.09,12
7,Alex Caffi,75.0,20,73.33,22
8,Alexander Albon,105.0,13,87.62,28
9,Alexander Wurz,69.0,10,85.51,24


In [102]:
# Get all columns related to Health Problems
columns_health = ['Illness', 'Injured', 'Injury', 'Driver unwell','Exhaust', 'Eye injury','Physical']

# Filter just the previous columns
status_health = results_with_status[columns_health]

# Sum all columns about health to count how much problems they had along their careers
status_health['Total Health'] = status_health[columns_health].sum(axis=1)

# Drop all the previous health columns
status_health = status_health.drop(columns=columns_health)

# Merge the previous dataset with race_counts
race_counts = race_counts.merge(status_health, on='Name', how='left')

C:\Users\Joaquim Meruje\AppData\Local\Temp\ipykernel_36732\2326342642.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  status_health['Total Health'] = status_health[columns_health].sum(axis=1)


In [103]:
# Download Dataframe
race_counts.to_csv('race_counts.csv', index=False)

race_counts.head(10)

,Name,Total Races,Total DNFs,Finished Races (%),Total + Laps,Total Health
0,Adrian Sutil,128.0,17,86.72,49,0
1,Aguri Suzuki,88.0,20,77.27,22,0
2,Alain Prost,202.0,27,86.63,18,0
3,Alan Jones,117.0,27,76.92,29,2
4,Alberto Ascari,36.0,9,75.00,5,0
5,Alessandro Nannini,77.0,25,67.53,12,1
6,Alessandro Zanardi,44.0,18,59.09,12,1
7,Alex Caffi,75.0,20,73.33,22,0
8,Alexander Albon,105.0,13,87.62,28,0
9,Alexander Wurz,69.0,10,85.51,24,0


### 1.2) Mechanical Problems Evolution

In [104]:
# Define group of coluns

engine_columns = [
    "Alternator", "Battery", "Cooling system",
    "Crankshaft", "ERS", "Engine", "Engine fire", 
    "Engine misfire", "Magneto", "Overheating",
    "Power Unit", "Power loss", "Radiator", 
    "Spark plugs", "Supercharger", "Turbo"
] 

aerodynamics_columns = [
    "Broken wing", "Chassis", "Front wing", 
    "Rear wing", "Undertray", "Underweight"
]

transmission_columns = [
    "Axle", "CV joint", "Differential", 
    "Driveshaft", "Drivetrain", "Gearbox", 
    "Halfshaft", "Transmission"
]

electronic_systems_columns = [
    "Electrical", "Electronics", "Ignition", "Launch control"
]

mechanical_systems_columns = [
    "Clutch", "Hydraulics", "Mechanical", "Pneumatics"
]

suspension_and_direction_columns = [
    "Handling", "Steering", "Suspension", "Track rod"
]

fuel_columns = [
    "Fuel", "Fuel leak", "Fuel pipe", "Fuel pressure", 
    "Fuel pump", "Fuel rig", "Fuel system", 
    "Out of fuel", "Refuelling"
]

oil_columns = [
    "Oil leak", "Oil line", "Oil pipe", 
    "Oil pressure", "Oil pump"
]

water_columns = [
    "Water leak", "Water pipe", "Water pressure", "Water pump"
]

wheel_columns = [
    "Wheel", "Wheel bearing", "Wheel nut", "Wheel rim"
]

In [105]:
# Merge results dataframe with status problems
status_by_races = results.merge(status[['statusId', 'status']], on='statusId', how='right')
status_by_races = status_by_races.groupby('raceId')['status'].apply(', '.join).reset_index()

# Merge previous dataframe with year of each race 
status_by_races = status_by_races.merge(races[['raceId', 'year']], on='raceId', how='right')
status_by_races = status_by_races.dropna()

In [106]:
# Get together all the previous groups definition
mechanical_parameters = (
    engine_columns +
    aerodynamics_columns +
    transmission_columns +
    electronic_systems_columns +
    mechanical_systems_columns +
    suspension_and_direction_columns +
    fuel_columns +
    oil_columns +
    water_columns +
    wheel_columns
)

# Create new columns for each previous parameters
for coluna in mechanical_parameters:
    status_by_races[coluna] = 0

# Create new columns to count status parameters by each status
for parameters in mechanical_parameters:
    status_by_races[parameters] = status_by_races["status"].apply(lambda x: x.split(", ").count(parameters))

# Delete the previous column
status_by_races = status_by_races.drop(columns=['status'])

# Create a new column to compute the Total Number of Problems
status_by_races["Total Problems"] = status_by_races[mechanical_parameters].sum(axis=1)

In [107]:
# Group the dataframe by year
status_by_races_by_year = status_by_races.groupby('year').sum()
status_by_races_by_year = status_by_races_by_year.reset_index()

# Create new columns to sum the parameters status values by each mechanical group
status_by_races_by_year["engine_columns"] = status_by_races_by_year[engine_columns].sum(axis=1) 
status_by_races_by_year["aerodynamics_columns"] = status_by_races_by_year[aerodynamics_columns].sum(axis=1) 
status_by_races_by_year["transmission_columns"] = status_by_races_by_year[transmission_columns].sum(axis=1) 
status_by_races_by_year["electronic_systems_columns"] = status_by_races_by_year[electronic_systems_columns].sum(axis=1) 
status_by_races_by_year["mechanical_systems_columns"] = status_by_races_by_year[mechanical_systems_columns].sum(axis=1) 
status_by_races_by_year["suspension_and_direction_columns"] = status_by_races_by_year[suspension_and_direction_columns].sum(axis=1) 
status_by_races_by_year["fuel_columns"] = status_by_races_by_year[fuel_columns].sum(axis=1) 
status_by_races_by_year["oil_columns"] = status_by_races_by_year[oil_columns].sum(axis=1) 
status_by_races_by_year["water_columns"] = status_by_races_by_year[water_columns].sum(axis=1) 
status_by_races_by_year["wheel_columns"] = status_by_races_by_year[wheel_columns].sum(axis=1) 

status_by_races_by_year.head(10)

,year,raceId,Alternator,Battery,Cooling system,Crankshaft,ERS,Engine,Engine fire,Engine misfire,...,engine_columns,aerodynamics_columns,transmission_columns,electronic_systems_columns,mechanical_systems_columns,suspension_and_direction_columns,fuel_columns,oil_columns,water_columns,wheel_columns
0,1950,5852.0,0,0,0,0,0,19,0,0,...,27,0,11,0,1,1,2,15,1,1
1,1951,6628.0,0,0,0,0,0,30,0,0,...,42,0,12,6,3,0,2,6,0,1
2,1952,6564.0,0,0,0,0,0,20,0,0,...,28,0,17,4,3,4,1,6,0,2
3,1953,7308.0,0,0,0,0,0,30,0,0,...,38,0,24,5,7,3,6,6,0,4
4,1954,7227.0,0,0,0,0,0,34,0,0,...,34,0,15,1,5,7,8,7,0,4
5,1955,5565.0,0,0,0,0,0,15,0,0,...,18,0,23,1,1,3,10,5,0,1
6,1956,6300.0,0,0,0,0,0,24,0,0,...,27,1,16,3,2,7,6,5,0,1
7,1957,6236.0,0,0,0,0,0,19,0,0,...,23,1,8,1,6,5,4,5,2,2
8,1958,8470.0,0,0,0,0,0,30,0,0,...,42,0,14,1,3,7,7,2,0,0
9,1959,6840.0,0,0,0,0,0,16,0,0,...,23,0,20,2,7,1,1,3,0,1


In [108]:
mechanical_columns = (
    "engine_columns",
    "aerodynamics_columns",
    "transmission_columns",
    "electronic_systems_columns",
    "mechanical_systems_columns",
    "suspension_and_direction_columns",
    "fuel_columns",
    "oil_columns",
    "water_columns",
    "wheel_columns"
)

# Drop the parameters status columns
mechanical_parameters_df = status_by_races_by_year.drop(columns=mechanical_parameters)

# Rename each mechanical group column
mechanical_parameters_df.rename(columns={
    'engine_columns': 'Engine',
    'aerodynamics_columns': 'Aerodynamics',
    'transmission_columns': 'Transmission',
    'electronic_systems_columns': 'Eletronics',
    'mechanical_systems_columns': 'Mechanical',
    'suspension_and_direction_columns': 'Suspension and Direction',
    'fuel_columns': 'Fuel',
    'oil_columns': 'Oil',
    'water_columns': 'Water',
    'wheel_columns': 'Wheel'
}, inplace=True)

# Download Dataframe
mechanical_parameters_df.to_csv('race_problems_years.csv', index=False)

mechanical_parameters_df.head(10)


,year,raceId,Total Problems,Engine,Aerodynamics,Transmission,Eletronics,Mechanical,Suspension and Direction,Fuel,Oil,Water,Wheel
0,1950,5852.0,59,27,0,11,0,1,1,2,15,1,1
1,1951,6628.0,72,42,0,12,6,3,0,2,6,0,1
2,1952,6564.0,65,28,0,17,4,3,4,1,6,0,2
3,1953,7308.0,93,38,0,24,5,7,3,6,6,0,4
4,1954,7227.0,81,34,0,15,1,5,7,8,7,0,4
5,1955,5565.0,62,18,0,23,1,1,3,10,5,0,1
6,1956,6300.0,68,27,1,16,3,2,7,6,5,0,1
7,1957,6236.0,57,23,1,8,1,6,5,4,5,2,2
8,1958,8470.0,76,42,0,14,1,3,7,7,2,0,0
9,1959,6840.0,58,23,0,20,2,7,1,1,3,0,1


### 1.3) Status Results for Teams

In [109]:
# Merge results dataframe with status problems
status_by_races = results.merge(status[['statusId', 'status']], on='statusId', how='right')

# Merge previous dataframe with year of each race 
status_by_races = status_by_races.merge(races[['raceId', 'year']], on='raceId', how='right')

# Merge previous dataframe with constructor names and countries
status_by_races = status_by_races.merge(constructors[['constructorId', 'name', 'nationality']], on='constructorId', how='right')
status_by_races.rename(columns={'name': 'Team Name'}, inplace=True)

In [110]:
# Merge datasets to get drivers names
status_by_races = status_by_races.merge(drivers[['driverId', 'forename', 'surname']], on='driverId', how='right')

# Create a new column to have the complete names
status_by_races['Name'] = status_by_races['forename'] + ' ' + status_by_races['surname']
status_by_races = status_by_races.drop(columns=['forename', 'surname'])

# Drop unnecessary columns 
status_by_races = status_by_races.drop(columns=['fastestLapTime', 'fastestLapSpeed', 'positionText', 'positionOrder', 'points'])
status_by_races = status_by_races.drop(columns=['time', 'milliseconds', 'fastestLap', 'number', 'grid', 'position', 'laps', 'rank'])

# Delete null values
status_by_races = status_by_races.dropna()

status_by_races.head(10)

,resultId,raceId,driverId,constructorId,statusId,status,year,Team Name,nationality,Name
0,7573.0,1.0,1.0,1.0,2.0,Disqualified,2009.0,McLaren,British,Lewis Hamilton
1,7580.0,2.0,1.0,1.0,1.0,Finished,2009.0,McLaren,British,Lewis Hamilton
2,7599.0,3.0,1.0,1.0,1.0,Finished,2009.0,McLaren,British,Lewis Hamilton
3,7617.0,4.0,1.0,1.0,1.0,Finished,2009.0,McLaren,British,Lewis Hamilton
4,7642.0,5.0,1.0,1.0,11.0,+1 Lap,2009.0,McLaren,British,Lewis Hamilton
5,7665.0,6.0,1.0,1.0,11.0,+1 Lap,2009.0,McLaren,British,Lewis Hamilton
6,7686.0,7.0,1.0,1.0,1.0,Finished,2009.0,McLaren,British,Lewis Hamilton
7,7709.0,8.0,1.0,1.0,11.0,+1 Lap,2009.0,McLaren,British,Lewis Hamilton
8,7731.0,9.0,1.0,1.0,11.0,+1 Lap,2009.0,McLaren,British,Lewis Hamilton
9,7734.0,10.0,1.0,1.0,1.0,Finished,2009.0,McLaren,British,Lewis Hamilton


In [111]:
# Teams evolution

redbull = ['Stewart', 'Jaguar', 'Red Bull']
ferrari = ['Ferrari']
alpine = ['Toleman', 'Benetton', 'Renault', 'Lotus', 'Renault', 'Alpine F1 Team']
mercedes = ['Tyrrell', 'BAR', 'Honda', 'Brawn', 'Mercedes']
mclaren = ['McLaren']
rb = ['Minardi', 'Toro Rosso', 'AlphaTauri', 'RB F1 Team']
sauber = ['Sauber', 'BMW Sauber', 'Sauber', 'Alfa Romeo', 'Sauber']
haas = ['Haas F1 Team']
aston_martin = ['Jordan', 'MF1', 'Spyker', 'Force India', 'Racing Point', 'Aston Martin'] 
williams = ['Williams']

manor = ['Virgin', 'Marussia', 'Manor Marussia']
caterham = ['Lotus', 'Caterham']
hispania = ['HRT']

arrows = ['Arrows','Footwork']
prost = ['Ligier', 'Prost']
toyota = ['Toyota']
super_aguri = ['Super Aguri']

In [112]:
# Search for teams in each previous list

team = caterham
existing = status_by_races['Team Name'].isin(team)

# Values that exist in the column
existing_values = status_by_races.loc[existing, 'Team Name'].unique()

# Values that DO NOT exist in the column
non_existing_values = list(set(team) - set(existing_values))

print("Values that exist in the column:", existing_values)
print("Values that do not exist in the column:", non_existing_values)

Values that exist in the column: ['Lotus' 'Caterham']
Values that do not exist in the column: []


In [113]:
# Search for the years that a team was active
years_team = status_by_races[status_by_races['Team Name'] == 'Caterham']['year'].unique()
print(sorted(years_team))

[2012.0, 2013.0, 2014.0]


In [114]:
# Filtrar teams by each name and count the ocurrences
names_filtered = status_by_races[status_by_races['Team Name'].str.startswith('Mer', na=False)]['Team Name']

#print(names_filtered.head(30))
print(names_filtered.count())


684


In [115]:
# Create a list for all teams
teams_list = (redbull + ferrari + alpine + mercedes + mclaren + rb + sauber + haas + caterham + 
              aston_martin + williams + manor + hispania + arrows + prost + toyota + super_aguri)

status_by_races = status_by_races[status_by_races['Team Name'].isin(teams_list)]

# Download Dataframe
status_by_races.to_csv('teams_status.csv', index=False)

In [116]:
status_by_races

,resultId,raceId,driverId,constructorId,statusId,status,year,Team Name,nationality,Name
0,7573.0,1.0,1.0,1.0,2.0,Disqualified,2009.0,McLaren,British,Lewis Hamilton
1,7580.0,2.0,1.0,1.0,1.0,Finished,2009.0,McLaren,British,Lewis Hamilton
2,7599.0,3.0,1.0,1.0,1.0,Finished,2009.0,McLaren,British,Lewis Hamilton
3,7617.0,4.0,1.0,1.0,1.0,Finished,2009.0,McLaren,British,Lewis Hamilton
4,7642.0,5.0,1.0,1.0,11.0,+1 Lap,2009.0,McLaren,British,Lewis Hamilton
...,...,...,...,...,...,...,...,...,...,...
26754,26701.0,1141.0,861.0,3.0,3.0,Accident,2024.0,Williams,British,Franco Colapinto
26755,26718.0,1142.0,861.0,3.0,1.0,Finished,2024.0,Williams,British,Franco Colapinto
26756,26743.0,1143.0,861.0,3.0,4.0,Collision,2024.0,Williams,British,Franco Colapinto
26757,26763.0,1144.0,861.0,3.0,5.0,Engine,2024.0,Williams,British,Franco Colapinto


### 1.3) Points lost from the starting grid in the event of a withdrawal

In [117]:
# Function to return the number of points based on the final position

def compute_points(grid_position, year):

    if year < 1960:
        if grid_position == 1:
            return(8)
        elif grid_position == 2:
            return(6)
        elif grid_position == 3:
            return(4)
        elif grid_position == 4:
            return(3)
        elif grid_position == 5:
            return(2)
        else:
            return(0)

    elif year >= 1960 and year <= 1990:
        if grid_position == 1:
            return(9)
        elif grid_position == 2:
            return(6)
        elif grid_position == 3:
            return(4)
        elif grid_position == 4:
            return(3)
        elif grid_position == 5:
            return(2)
        elif grid_position == 6:
            return(1)
        else:
            return(0)
        
    elif year > 1990 and year <= 2002:
        if grid_position == 1:
            return(10)
        elif grid_position == 2:
            return(6)
        elif grid_position == 3:
            return(4)
        elif grid_position == 4:
            return(3)
        elif grid_position == 5:
            return(2)
        elif grid_position == 6:
            return(1)   
        else:
            return(0) 
        
    elif year > 2002 and year <= 2009:
        if grid_position == 1:
            return(10)
        elif grid_position == 2:
            return(8)
        elif grid_position == 3:
            return(6)
        elif grid_position == 4:
            return(5)
        elif grid_position == 5:
            return(4)
        elif grid_position == 6:
            return(3)  
        elif grid_position == 7:
            return(2) 
        elif grid_position == 8:
            return(1) 
        else:
            return(0)

    elif year > 2009:
        if grid_position == 1:
            return(25)
        elif grid_position == 2:
            return(18)
        elif grid_position == 3:
            return(15)
        elif grid_position == 4:
            return(12)
        elif grid_position == 5:
            return(10)
        elif grid_position == 6:
            return(8)  
        elif grid_position == 7:
            return(6) 
        elif grid_position == 8:
            return(4)     
        elif grid_position == 9:
            return(2) 
        elif grid_position == 10:
            return(1) 
        else:
            return(0)

In [118]:
# Merge dataframes to include grid position
status_by_races = status_by_races.merge(results[['resultId', 'grid']], on='resultId', how='left')

# Compute points lost based on the grid position 
status_by_races['Points Lost'] = status_by_races.apply(
    lambda row: 0 if (row['status'] == 'Finished' or 'Lap' in row['status']) 
    else compute_points(row['grid'], row['year']),
    axis=1
)

# Download Dataframe
status_by_races.to_csv('teams_points_lost.csv', index=False)

In [119]:
status_by_races

,resultId,raceId,driverId,constructorId,statusId,status,year,Team Name,nationality,Name,grid,Points Lost
0,7573.0,1.0,1.0,1.0,2.0,Disqualified,2009.0,McLaren,British,Lewis Hamilton,18,0
1,7580.0,2.0,1.0,1.0,1.0,Finished,2009.0,McLaren,British,Lewis Hamilton,12,0
2,7599.0,3.0,1.0,1.0,1.0,Finished,2009.0,McLaren,British,Lewis Hamilton,9,0
3,7617.0,4.0,1.0,1.0,1.0,Finished,2009.0,McLaren,British,Lewis Hamilton,5,0
4,7642.0,5.0,1.0,1.0,11.0,+1 Lap,2009.0,McLaren,British,Lewis Hamilton,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...
17630,26701.0,1141.0,861.0,3.0,3.0,Accident,2024.0,Williams,British,Franco Colapinto,16,0
17631,26718.0,1142.0,861.0,3.0,1.0,Finished,2024.0,Williams,British,Franco Colapinto,0,0
17632,26743.0,1143.0,861.0,3.0,4.0,Collision,2024.0,Williams,British,Franco Colapinto,19,0
17633,26763.0,1144.0,861.0,3.0,5.0,Engine,2024.0,Williams,British,Franco Colapinto,20,0


## 2) Teams and Drivers

In [120]:
results

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22.0,1,1.0,1,1,10.0,58,1:34:50.616,5690616.0,39.0,2.0,1:27.452,218.300,1
1,2,18,2,2,3.0,5,2.0,2,2,8.0,58,+5.478,5696094.0,41.0,3.0,1:27.739,217.586,1
2,3,18,3,3,7.0,7,3.0,3,3,6.0,58,+8.163,5698779.0,41.0,5.0,1:28.090,216.719,1
3,4,18,4,4,5.0,11,4.0,4,4,5.0,58,+17.181,5707797.0,58.0,7.0,1:28.603,215.464,1
4,5,18,5,1,23.0,3,5.0,5,5,4.0,58,+18.014,5708630.0,43.0,1.0,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26754,26760,1144,825,210,20.0,14,16.0,16,16,0.0,57,NaN,NaN,57.0,1.0,1:25.637,222.002,11
26755,26761,1144,859,215,30.0,12,17.0,17,17,0.0,55,NaN,NaN,52.0,12.0,1:28.751,214.212,5
26756,26762,1144,822,15,77.0,9,NaN,R,18,0.0,30,NaN,NaN,14.0,19.0,1:29.482,212.462,130
26757,26763,1144,861,3,43.0,20,NaN,R,19,0.0,26,NaN,NaN,5.0,17.0,1:29.411,212.631,5


In [121]:
# Merge status problems name with status id 

results_by_driver = results.merge(drivers[['driverId', 'forename', 'surname', 'dob', 'nationality']], on='driverId', how='right')

# Create a new column to have the complete names

results_by_driver ['name'] = results_by_driver['forename'] + ' ' + results_by_driver['surname']
results_by_driver  = results_by_driver.drop(columns=['forename', 'surname'])

# Download Dataframe
results_by_driver.to_csv('drivers.csv', index=False)

In [122]:
results_by_driver

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,dob,nationality,name
0,1,18,1,1,22.0,1,1.0,1,1,10.0,...,1:34:50.616,5690616.0,39.0,2.0,1:27.452,218.300,1,1985-01-07,British,Lewis Hamilton
1,27,19,1,1,22.0,9,5.0,5,5,4.0,...,+46.548,5525103.0,53.0,3.0,1:35.462,209.033,1,1985-01-07,British,Lewis Hamilton
2,57,20,1,1,22.0,3,13.0,13,13,0.0,...,NaN,NaN,25.0,19.0,1:35.520,203.969,11,1985-01-07,British,Lewis Hamilton
3,69,21,1,1,22.0,5,3.0,3,3,6.0,...,+4.187,5903238.0,20.0,3.0,1:22.017,204.323,1,1985-01-07,British,Lewis Hamilton
4,90,22,1,1,22.0,3,2.0,2,2,8.0,...,+3.779,5213230.0,31.0,2.0,1:26.529,222.085,1,1985-01-07,British,Lewis Hamilton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26754,26701,1141,861,3,43.0,16,NaN,R,17,0.0,...,NaN,NaN,21.0,18.0,1:24.296,184.022,3,2003-05-27,Argentinian,Franco Colapinto
26755,26718,1142,861,3,43.0,0,14.0,14,14,0.0,...,+1:15.172,5001141.0,37.0,15.0,1:36.867,230.456,1,2003-05-27,Argentinian,Franco Colapinto
26756,26743,1143,861,3,43.0,19,NaN,R,19,0.0,...,NaN,NaN,NaN,0.0,NaN,NaN,4,2003-05-27,Argentinian,Franco Colapinto
26757,26763,1144,861,3,43.0,20,NaN,R,19,0.0,...,NaN,NaN,5.0,17.0,1:29.411,212.631,5,2003-05-27,Argentinian,Franco Colapinto


In [123]:
constructors.rename(columns={'name': 'Team Name'}, inplace=True)
constructors.rename(columns={'nationality': 'Team nationality'}, inplace=True)

results_by_team = results.merge(constructors[['constructorId', 'Team Name', 'Team nationality']], on='constructorId', how='right')

results_by_team = results_by_team.merge(drivers[['driverId', 'forename', 'surname', 'dob', 'nationality']], on='driverId', how='right')

# Create a new column to have the complete names

results_by_team ['name'] = results_by_team['forename'] + ' ' + results_by_team['surname']
results_by_team  = results_by_team.drop(columns=['forename', 'surname'])

In [124]:
# Merge previous dataframe with year of each race 
results_by_team = results_by_team.merge(races[['raceId', 'year']], on='raceId', how='right')

results_by_team  = results_by_team.drop(columns=['driverId', 'constructorId', 'positionText', 'statusId'])

# Download Dataframe
results_by_team.to_csv('teams.csv', index=False)

In [125]:
results_by_team

,resultId,raceId,number,grid,position,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,Team Name,Team nationality,dob,nationality,name,year
0,7573.0,1.0,1.0,18.0,NaN,20.0,0.0,58.0,NaN,NaN,39.0,13.0,1:29.020,214.455,McLaren,British,1985-01-07,British,Lewis Hamilton,2009
1,7563.0,1.0,6.0,9.0,10.0,10.0,0.0,58.0,+7.085,5662869.0,48.0,5.0,1:28.283,216.245,BMW Sauber,German,1977-05-10,German,Nick Heidfeld,2009
2,7559.0,1.0,16.0,5.0,6.0,6.0,3.0,58.0,+5.722,5661506.0,48.0,1.0,1:27.706,217.668,Williams,British,1985-06-27,German,Nico Rosberg,2009
3,7558.0,1.0,7.0,10.0,5.0,5.0,4.0,58.0,+4.879,5660663.0,53.0,9.0,1:28.712,215.199,Renault,French,1981-07-29,Spanish,Fernando Alonso,2009
4,7572.0,1.0,2.0,12.0,NaN,19.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,McLaren,British,1981-10-19,Finnish,Heikki Kovalainen,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26754,26757.0,1144.0,24.0,15.0,13.0,13.0,0.0,57.0,NaN,NaN,56.0,8.0,1:27.982,216.085,Sauber,Swiss,1999-05-30,Chinese,Guanyu Zhou,2024
26755,26754.0,1144.0,81.0,2.0,10.0,10.0,1.0,58.0,+1:23.821,5277112.0,51.0,4.0,1:27.690,216.804,McLaren,British,2001-04-06,Australian,Oscar Piastri,2024
26756,26761.0,1144.0,30.0,12.0,17.0,17.0,0.0,55.0,NaN,NaN,52.0,12.0,1:28.751,214.212,RB F1 Team,Italian,2002-02-11,New Zealander,Liam Lawson,2024
26757,26763.0,1144.0,43.0,20.0,NaN,19.0,0.0,26.0,NaN,NaN,5.0,17.0,1:29.411,212.631,Williams,British,2003-05-27,Argentinian,Franco Colapinto,2024


In [126]:
# Create a new dataframe to get the final results of each Constructors Championships by year

# Merge dataframes to get team names and data of each GP
constructors_data = constructors_standings.merge(constructors[['constructorId', 'Team Name']], on='constructorId', how='right')
constructors_data = constructors_data.merge(races[['raceId', 'date']], on='raceId', how='right')

# Convert date column and create a year column
constructors_data['date'] = pd.to_datetime(constructors_data['date'])
constructors_data['year'] = constructors_data['date'].dt.year

# Identify the last race of each year and filter the constructors_data dataset
latest_races = constructors_data.groupby('year')['date'].max().reset_index()
constructors_data = constructors_data.merge(latest_races, on=['year', 'date'], how='inner')

In [127]:
# Drop some columns
constructors_data  = constructors_data.drop(columns=['constructorStandingsId', 'raceId'])

# Delete NaN values
constructors_data = constructors_data.dropna()

# Sort dataframe
constructors_data = constructors_data.sort_values(by=['year', 'points'], ascending=[True, False])

# Download Dataframe
constructors_data.to_csv('constructors_championships.csv', index=False)

In [128]:
constructors_data

,constructorId,points,position,positionText,wins,Team Name,date,year
770,118.0,48.0,1.0,1,6.0,Vanwall,1958-10-19,1958
764,6.0,40.0,2.0,2,2.0,Ferrari,1958-10-19,1958
767,87.0,31.0,3.0,3,2.0,Cooper,1958-10-19,1958
766,66.0,18.0,4.0,4,0.0,BRM,1958-10-19,1958
769,105.0,6.0,5.0,5,0.0,Maserati,1958-10-19,1958
...,...,...,...,...,...,...,...,...
926,214.0,65.0,6.0,6,0.0,Alpine F1 Team,2024-12-08,2024
925,210.0,58.0,7.0,7,0.0,Haas F1 Team,2024-12-08,2024
927,215.0,46.0,8.0,8,0.0,RB F1 Team,2024-12-08,2024
919,3.0,17.0,9.0,9,0.0,Williams,2024-12-08,2024


In [ ]:
# Create a new dataframe to get the final results of each Drivers Championships by year

# Merge dataframes to get team names and data of each GP
drivers_championships = driver_standings.merge(drivers[['driverId', 'forename', 'surname']], on='driverId', how='right')

# Create a new column to have the complete names
drivers_championships['driverName'] = drivers_championships['forename'] + ' ' + drivers_championships['surname']
drivers_championships  = drivers_championships.drop(columns=['forename', 'surname'])

# Merge dataframes to get the team by each driver at each race
drivers_championships = drivers_championships.merge(results[['raceId', 'driverId', 'constructorId']], on=['raceId', 'driverId'], how='right')
drivers_championships = drivers_championships.merge(constructors[['constructorId', 'Team Name']], on='constructorId', how='right')

# Merge dataframes to get the date of each GP
drivers_championships  = drivers_championships.merge(races[['raceId', 'date']], on='raceId', how='right')

# Convert date column and create a year column
drivers_championships['date'] = pd.to_datetime(drivers_championships['date'])
drivers_championships['year'] = drivers_championships['date'].dt.year

# Identify the last race of each year and filter the constructors_data dataset
latest_races = drivers_championships.groupby('year')['date'].max().reset_index()
drivers_championships = drivers_championships.merge(latest_races, on=['year', 'date'], how='inner')

In [130]:
# Drop some columns
drivers_championships = drivers_championships.drop(columns=['driverStandingsId', 'constructorId']) 
drivers_championships.rename(columns={'name': 'teamName'}, inplace=True)

# Delete NaN values
drivers_championships = drivers_championships.dropna()

# Sort dataframe
drivers_championships = drivers_championships.sort_values(by=['year', 'points'], ascending=[True, False])

# Download Dataframe
drivers_championships.to_csv('drivers_championships.csv', index=False)

In [131]:
drivers_championships

,raceId,driverId,points,position,positionText,wins,driverName,Team Name,date,year
1468,839.0,642.0,30.0,1.0,1,3.0,Nino Farina,Alfa Romeo,1950-09-03,1950
1471,839.0,579.0,27.0,2.0,2,3.0,Juan Fangio,Alfa Romeo,1950-09-03,1950
1473,839.0,579.0,27.0,2.0,2,3.0,Juan Fangio,Alfa Romeo,1950-09-03,1950
1469,839.0,786.0,24.0,3.0,3,0.0,Luigi Fagioli,Alfa Romeo,1950-09-03,1950
1484,839.0,627.0,13.0,4.0,4,0.0,Louis Rosier,Talbot-Lago,1950-09-03,1950
...,...,...,...,...,...,...,...,...,...,...
1767,1144.0,861.0,5.0,19.0,19,0.0,Franco Colapinto,Williams,2024-12-08,2024
1772,1144.0,855.0,4.0,20.0,20,0.0,Guanyu Zhou,Sauber,2024-12-08,2024
1783,1144.0,859.0,4.0,21.0,21,0.0,Liam Lawson,RB F1 Team,2024-12-08,2024
1773,1144.0,822.0,0.0,22.0,22,0.0,Valtteri Bottas,Sauber,2024-12-08,2024


In [132]:
# Create a new dataframe to merge with drivers_championships dataframe and have more info

# Merge dataframes to get team names and data of each GP
drivers_championships_v2 = results.merge(drivers[['driverId', 'forename', 'surname']], on='driverId', how='right')

# Create a new column to have the complete names
drivers_championships_v2['driverName'] = drivers_championships_v2['forename'] + ' ' + drivers_championships_v2['surname']
drivers_championships_v2  = drivers_championships_v2.drop(columns=['forename', 'surname'])

# Merge dataframes to get the team by each driver at each race
drivers_championships_v2 = drivers_championships_v2.merge(constructors[['constructorId', 'Team Name']], on='constructorId', how='right')

# Merge dataframes to get the date of each GP
drivers_championships_v2 = drivers_championships_v2.merge(races[['raceId', 'date']], on='raceId', how='right')

# Convert date column and create a year column
drivers_championships_v2['date'] = pd.to_datetime(drivers_championships_v2['date'])
drivers_championships_v2['year'] = drivers_championships_v2['date'].dt.year

In [133]:
# Compute new columns to get podiums, victories and pole positions
drivers_championships_v2['podium'] = drivers_championships_v2['position'].apply(lambda x: 1 if x in [1.0, 2.0, 3.0] else 0)
drivers_championships_v2['victory'] = drivers_championships_v2['position'].apply(lambda x: 1 if x == 1 else 0)
drivers_championships_v2['pole_position'] = drivers_championships_v2['grid'].apply(lambda x: 1 if x == 1 else 0)
drivers_championships_v2['race_entries'] = drivers_championships_v2.groupby(['year', 'driverId'])['driverId'].transform('count')

# Groupby the previus dataframe to get maximum speed, total number of points and total number of poles, podiumns and victories
drivers_championships_v2 = drivers_championships_v2.groupby(['year', 'driverId', 'driverName', 'Team Name', 'race_entries']).agg({
    'fastestLapSpeed': 'max',  
    'points': 'sum',           
    'pole_position': 'sum',    
    'podium': 'sum',         
    'victory': 'sum'           
}).reset_index()

In [134]:
drivers_championships_v2

,year,driverId,driverName,Team Name,race_entries,fastestLapSpeed,points,pole_position,podium,victory
0,1950,427.0,Maurice Trintignant,Simca,2,NaN,0.0,0,0,0
1,1950,498.0,José Froilán González,Maserati,2,NaN,0.0,0,0,0
2,1950,501.0,Harry Schell,Cooper,2,NaN,0.0,0,0,0
3,1950,501.0,Harry Schell,Talbot-Lago,2,NaN,0.0,0,0,0
4,1950,509.0,Jim Rathmann,Wetteroth,1,NaN,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
3548,2024,859.0,Liam Lawson,RB F1 Team,6,230.187,4.0,0,0,0
3549,2024,860.0,Oliver Bearman,Ferrari,3,241.103,6.0,0,0,0
3550,2024,860.0,Oliver Bearman,Haas F1 Team,3,201.879,1.0,0,0,0
3551,2024,861.0,Franco Colapinto,Williams,9,249.077,5.0,0,0,0


In [135]:
# Merge the drivers_championships dataframes
drivers_championships_v3 = drivers_championships_v2.merge(
    drivers_championships[['driverId', 'year', 'positionText']],
    on=['driverId', 'year'],
    how='left'
)

# Iterate through each year to check for NaN values and multiple teams for the same driver
for year, year_df in drivers_championships_v3.groupby('year'):
    
    # Filter rows with NaN in 'positionText'
    year_nan_df = year_df[year_df['positionText'].isna()]
    
    # Check for drivers with multiple teams in the same year
    multi_teams_drivers = year_nan_df.groupby('driverId')['Team Name'].nunique()
    
    # Filter drivers who have more than one unique team in that year
    multi_teams_drivers = multi_teams_drivers[multi_teams_drivers > 1].index
    
    if len(multi_teams_drivers) > 0:
        
        # Filter the dataframe to get only the rows for drivers who competed for multiple teams
        multi_teams_df = year_nan_df[year_nan_df['driverId'].isin(multi_teams_drivers)]
        
        # Group by 'driverId' and aggregate the results
        aggregated_df = multi_teams_df.groupby('driverId').agg(
            total_points=('points', 'sum'),
            teams=('Team Name', lambda x: ', '.join(sorted(x.unique())))  # Concatenate teams with commas
        ).reset_index()
        
        # Update the original DataFrame explicitly
        for _, row in aggregated_df.iterrows():
            driver_id = row['driverId']
            
            # Find the indices for the driver in the original DataFrame
            indices = drivers_championships_v3[(drivers_championships_v3['year'] == year) & (drivers_championships_v3['driverId'] == driver_id)].index
            
            # Update the first occurrence with the aggregated data
            if len(indices) > 0:
                drivers_championships_v3.loc[indices[0], 'points'] = row['total_points']
                drivers_championships_v3.loc[indices[0], 'Team Name'] = row['teams']
                
            # Drop the remaining rows for the driver
            drivers_championships_v3.drop(indices[1:], inplace=True)


In [136]:
# Iterate through each year and fill in missing positions
for year, year_df in drivers_championships_v3.groupby('year'):
    
    # Expected positions (from 1 to the number of entries in that year)
    expected_positions = set(range(1, len(year_df) + 1))
    
    # Present positions, excluding disqualified drivers (NaNs and letters like 'D')
    present_positions = set(year_df[year_df['positionText'] != 'D']['positionText'].dropna().astype(int))
    
    # Calculate missing positions
    missing_positions = sorted(expected_positions - present_positions)

    # Sort the DataFrame by points and filter rows with NaN in 'positionText'
    df_sorted = year_df[year_df['positionText'].isna()].sort_values(by='points', ascending=False).copy()

    # Assign the missing positions to the sorted DataFrame
    if len(missing_positions) >= len(df_sorted):
        df_sorted['positionText'] = sorted(missing_positions[:len(df_sorted)])

    # Check for disqualification ('D') and assign the last position
    if 'D' in year_df['positionText'].values:
        disqualified_index = year_df.loc[year_df['positionText'] == 'D'].index
        drivers_championships_v3.loc[disqualified_index, 'positionText'] = str(max(expected_positions))

    # Update the original DataFrame with the new positions
    drivers_championships_v3.update(df_sorted)

In [137]:
drivers_championships_v3[drivers_championships_v3['year'] == 2024]

,year,driverId,driverName,Team Name,race_entries,fastestLapSpeed,points,pole_position,podium,victory,positionText
3536,2024,1.0,Lewis Hamilton,Mercedes,24,255.849,207.0,0,5,2,7
3537,2024,4.0,Fernando Alonso,Aston Martin,24,251.432,70.0,0,0,0,9
3538,2024,807.0,Nico Hülkenberg,Haas F1 Team,24,250.432,36.0,0,0,0,11
3539,2024,815.0,Sergio Pérez,Red Bull,24,251.350,138.0,0,4,0,8
3540,2024,817.0,Daniel Ricciardo,RB F1 Team,18,247.625,7.0,0,0,0,17.0
3541,2024,822.0,Valtteri Bottas,Sauber,24,249.432,0.0,0,0,0,22
3542,2024,825.0,Kevin Magnussen,Haas F1 Team,22,249.946,14.0,0,0,0,15
3543,2024,830.0,Max Verstappen,Red Bull,24,255.120,399.0,8,14,9,1
3544,2024,832.0,Carlos Sainz,Ferrari,23,250.601,262.0,1,9,2,5
3545,2024,839.0,Esteban Ocon,Alpine F1 Team,23,247.261,23.0,0,1,0,14.0


In [138]:
# Rename Columns
drivers_championships_v3 = drivers_championships_v3.rename(columns={'positionText': 'final_position'})

# Convert type column
drivers_championships_v3['final_position'] = drivers_championships_v3['final_position'].astype(int)

# Sort dataframe
drivers_championships_v3 = drivers_championships_v3.sort_values(by=['year', 'final_position'], ascending=[True, True])

# Download Dataframe
drivers_championships_v3.to_csv('drivers_championships.csv', index=False)

In [139]:
# Get birthday and code name for eachd driver

drivers_data = drivers

# Merge dataframes to get team names and data of each GP and create a new column to have the complete names
drivers_data['driverName'] = drivers_data['forename'] + ' ' + drivers_data['surname']

# Drop columns and create a new csv file
drivers_data = drivers_data.drop(columns=['driverRef', 'forename', 'surname', 'url', 'forename', 'surname'])
drivers_data.to_csv('drivers_data.csv', index=False)

In [140]:
drivers_data

,driverId,number,code,dob,nationality,driverName
0,1,44.0,HAM,1985-01-07,British,Lewis Hamilton
1,2,NaN,HEI,1977-05-10,German,Nick Heidfeld
2,3,6.0,ROS,1985-06-27,German,Nico Rosberg
3,4,14.0,ALO,1981-07-29,Spanish,Fernando Alonso
4,5,NaN,KOV,1981-10-19,Finnish,Heikki Kovalainen
...,...,...,...,...,...,...
856,858,2.0,SAR,2000-12-31,American,Logan Sargeant
857,859,30.0,LAW,2002-02-11,New Zealander,Liam Lawson
858,860,38.0,BEA,2005-05-08,British,Oliver Bearman
859,861,43.0,COL,2003-05-27,Argentinian,Franco Colapinto


In [163]:
# Group dataset by year and team name to have drivers per team in each season
drivers_teams_by_season = drivers_championships_v3.groupby(['year', 'Team Name']).agg({
    'driverName': lambda x: list(x.unique())
}).reset_index()

# Rename the column frm driverName to drivers
drivers_teams_by_season.rename(columns={'driverName': 'drivers'}, inplace=True)

In [164]:
# Identify records where the Team Name is made up of multiple teams and remove those lines
drivers_championships_split = drivers_teams_by_season[drivers_teams_by_season['Team Name'].str.contains(',')]
drivers_teams_by_season = drivers_teams_by_season[~drivers_teams_by_season['Team Name'].str.contains(',')]

# Divide and reallocate drivers to each team
expanded_rows = []

for _, row in drivers_championships_split.iterrows():
    year = row['year']
    drivers = row['drivers']
    teams = row['Team Name'].split(', ')
    
    for team in teams:
        expanded_rows.append({'year': year, 'Team Name': team, 'drivers': drivers})

# Concat previous dataframes
expanded_df = pd.DataFrame(expanded_rows)
drivers_teams_by_season = pd.concat([drivers_teams_by_season, expanded_df], ignore_index=True)

In [165]:
# Filter duplicate entries for Oliver Bearman in 2024
bearman_entries = drivers_teams_by_season[(drivers_teams_by_season['year'] == 2024) & (drivers_teams_by_season['drivers'].apply(lambda x: 'Oliver Bearman' in x))]

# Ensure no duplication when adding Oliver Bearman to the driver lists
def merge_bearman(row):
    if row['Team Name'] == 'Ferrari' or row['Team Name'] == 'Haas F1 Team':
        row['drivers'] = list(set(row['drivers'] + ['Oliver Bearman']))
    return row

# Update the main DataFrame by merging Bearman's records
drivers_teams_by_season = drivers_teams_by_season.apply(merge_bearman, axis=1)

# Remove the individual entries for Oliver Bearman in Ferrari and Haas
drivers_teams_by_season = drivers_teams_by_season.drop(bearman_entries.index).reset_index(drop=True)

# Download csv file
drivers_teams_by_season.to_csv('drivers_teams_by_season.csv', index=False)

In [166]:
drivers_teams_by_season[drivers_teams_by_season['year'] == 2024]

,year,Team Name,drivers
1086,2024,Alpine F1 Team,"[Pierre Gasly, Esteban Ocon, Jack Doohan]"
1087,2024,Aston Martin,"[Fernando Alonso, Lance Stroll]"
1088,2024,Ferrari,"[Charles Leclerc, Carlos Sainz, Oliver Bearman]"
1089,2024,Haas F1 Team,"[Kevin Magnussen, Oliver Bearman, Nico Hülkenb..."
1090,2024,McLaren,"[Lando Norris, Oscar Piastri]"
1091,2024,Mercedes,"[George Russell, Lewis Hamilton]"
1092,2024,RB F1 Team,"[Yuki Tsunoda, Daniel Ricciardo, Liam Lawson]"
1093,2024,Red Bull,"[Max Verstappen, Sergio Pérez]"
1094,2024,Sauber,"[Guanyu Zhou, Valtteri Bottas]"
1095,2024,Williams,"[Alexander Albon, Franco Colapinto, Logan Sarg..."


## 2) Circuits

In [6]:
circuits

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park
...,...,...,...,...,...,...,...,...,...
72,75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,108,http://en.wikipedia.org/wiki/Algarve_Internati...
73,76,mugello,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190,255,http://en.wikipedia.org/wiki/Mugello_Circuit
74,77,jeddah,Jeddah Corniche Circuit,Jeddah,Saudi Arabia,21.63190,39.10440,15,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...
75,78,losail,Losail International Circuit,Al Daayen,Qatar,25.49000,51.45420,12,http://en.wikipedia.org/wiki/Losail_Internatio...


In [7]:
# Delete unnecessary columns
circuits_final = circuits.drop(columns=['circuitRef', 'url', 'lat', 'lng', 'alt'])

# Count number of races by eahc circuit
n_races_by_circuit = races.groupby('circuitId')['raceId'].count().reset_index()
n_races_by_circuit = n_races_by_circuit.rename(columns={'raceId': 'num_races'})
circuits_final = circuits_final.merge(n_races_by_circuit[['circuitId', 'num_races']], on='circuitId', how='right')

In [8]:
# Functions to convert lap times to seconds and back to string format

def convert_to_seconds(lap_str):
    try:
        m, s = lap_str.split(":")
        return int(m) * 60 + float(s)
    except:
        return None  
    
def convert_to_lap_str(seconds):
    if seconds is None:
        return None
    m = int(seconds // 60)
    s = seconds % 60
    return f"{m}:{s:06.3f}"  

# Convert fastestLapTime_sec to seconds for best_lap and merge with circuits
results['fastestLapTime_sec'] = results['fastestLapTime'].apply(convert_to_seconds)
results = results.merge(races[['raceId', 'circuitId']], on='raceId', how='left')

# Remove NaN values for fastestLapTime_sec
results_clean = results.dropna(subset=['fastestLapTime_sec'])

# Find the index of the fastest lap for each circuit and select the corresponding rows
idx_fast_lap = results_clean.groupby('circuitId')['fastestLapTime_sec'].idxmin()
best_lap = results_clean.loc[idx_fast_lap, ['circuitId', 'fastestLapTime_sec', 'fastestLapTime', 'driverId']].reset_index(drop=True)

In [9]:
# Merge with drivers to get driver names
best_lap = best_lap.merge(drivers[['driverId', 'forename', 'surname']], on='driverId', how='inner')
best_lap['driverName'] = best_lap['forename'] + ' ' + best_lap['surname']
best_lap = best_lap.drop(columns=['forename', 'surname'])

# Merge best lap times and driver name with circuits_final
circuits_final = circuits_final.merge(best_lap[['circuitId', 'fastestLapTime', 'driverName']], on='circuitId', how='right')

In [79]:
# Download csv file and print final dataset
circuits_final.to_csv('circuits_final.csv', index=False)
circuits_final.head(10)

,circuitId,name,location,country,num_races,fastestLapTime,driverName
0,1,Albert Park Grand Prix Circuit,Melbourne,Australia,27,1:19.813,Charles Leclerc
1,69,Circuit of the Americas,Austin,USA,12,1:36.169,Charles Leclerc
2,73,Baku City Circuit,Baku,Azerbaijan,8,1:43.009,Charles Leclerc
3,2,Sepang International Circuit,Kuala Lumpur,Malaysia,19,1:34.080,Sebastian Vettel
4,34,Circuit Paul Ricard,Le Castellet,France,18,1:32.740,Sebastian Vettel
5,35,Korean International Circuit,Yeongam County,Korea,4,1:39.605,Sebastian Vettel
6,68,Buddh International Circuit,Uttar Pradesh,India,3,1:27.249,Sebastian Vettel
7,3,Bahrain International Circuit,Sakhir,Bahrain,21,0:55.404,George Russell
8,4,Circuit de Barcelona-Catalunya,Montmeló,Spain,34,1:15.641,Giancarlo Fisichella
9,5,Istanbul Park,Istanbul,Turkey,9,1:24.770,Juan Pablo Montoya


In [29]:
# Create a new dataframe with the results of each race winner 
results_by_circuit = results[results['position'] == 1.0]
results_by_circuit = results_by_circuit.drop(columns=['number', 'position', 'positionText', 'positionOrder', 'points', 'statusId'])

# Merge results with circuits to get circuit names and locations
results_by_circuit = results_by_circuit.merge(circuits_final[['circuitId', 'name', 'location', 'country']], on='circuitId', how='inner')
results_by_circuit.rename(columns={'name': 'Circuit Name'}, inplace=True)

In [30]:
# Merge with drivers to get driver names
results_by_circuit = results_by_circuit.merge(drivers[['driverId', 'forename', 'surname']], on='driverId', how='inner')
results_by_circuit['driverName'] = results_by_circuit['forename'] + ' ' + results_by_circuit['surname']
results_by_circuit = results_by_circuit.drop(columns=['forename', 'surname'])

# Merge with constructors to get teams names
results_by_circuit = results_by_circuit.merge(constructors[['constructorId', 'name']], on='constructorId', how='inner')
results_by_circuit.rename(columns={'name': 'Team Name'}, inplace=True)

# Drop columns with IDs that are not needed and merge with races year
results_by_circuit = results_by_circuit.drop(columns=['driverId', 'constructorId', 'resultId', 'circuitId', 'rank'])
results_by_circuit = results_by_circuit.merge(races[['raceId', 'year']], on='raceId', how='inner')

In [31]:
# Calculate average fastest lap time and speed for each raceId
results_vel = results.groupby('raceId')[['fastestLapTime_sec', 'fastestLapSpeed']].mean().reset_index()
results_vel = results_vel.rename(columns={
    'fastestLapTime_sec': 'fastestLapTime_mean',
    'fastestLapSpeed': 'fastestLapSpeed_mean'
})

# Merge average fastest lap time and speed with results_by_circuit
results_by_circuit = results_by_circuit.merge(results_vel[['raceId', 'fastestLapTime_mean', 'fastestLapSpeed_mean']], on='raceId', how='inner')

In [47]:
# Download csv file and print final dataset
results_by_circuit['country'] = results_by_circuit['country'].replace('USA', 'United States')
results_by_circuit.to_csv('results_by_circuit.csv', index=False)
results_by_circuit.head(10)

,raceId,grid,laps,time,milliseconds,fastestLap,fastestLapTime,fastestLapSpeed,fastestLapTime_sec,Circuit Name,location,country,driverName,Team Name,year,fastestLapTime_mean,fastestLapSpeed_mean
0,18,1,58,1:34:50.616,5690616.0,39.0,1:27.452,218.300,87.452,Albert Park Grand Prix Circuit,Melbourne,Australia,Lewis Hamilton,McLaren,2008,89.100529,214.306647
1,343,2,58,1:28:47.620,5327620.0,54.0,1:30.075,213.342,90.075,Istanbul Park,Istanbul,Turkey,Lewis Hamilton,McLaren,2010,91.018042,211.209500
2,23,3,76,2:00:42.742,7242742.0,71.0,1:18.510,153.152,78.510,Circuit de Monaco,Monte-Carlo,Monaco,Lewis Hamilton,McLaren,2008,82.668200,146.499200
3,41,1,70,1:44:11.292,6251292.0,37.0,1:16.494,205.239,76.494,Circuit Gilles Villeneuve,Montreal,Canada,Lewis Hamilton,McLaren,2007,78.038571,201.214286
4,344,1,70,1:33:53.456,5633456.0,62.0,1:17.806,201.778,77.806,Circuit Gilles Villeneuve,Montreal,Canada,Lewis Hamilton,McLaren,2010,79.630696,197.242087
5,866,2,70,1:32:29.586,5549586.0,59.0,1:17.020,203.837,77.020,Circuit Gilles Villeneuve,Montreal,Canada,Lewis Hamilton,McLaren,2012,77.950917,201.489958
6,26,4,60,1:39:09.440,5949440.0,16.0,1:32.817,199.398,92.817,Silverstone Circuit,Silverstone,UK,Lewis Hamilton,McLaren,2008,93.756167,197.437222
7,27,1,67,1:31:20.874,5480874.0,17.0,1:16.039,216.552,76.039,Hockenheimring,Hockenheim,Germany,Lewis Hamilton,McLaren,2008,77.023300,213.797700
8,46,1,70,1:35:52.991,5752991.0,13.0,1:20.171,196.724,80.171,Hungaroring,Budapest,Hungary,Lewis Hamilton,McLaren,2007,81.507136,193.551727
9,10,4,70,1:38:23.876,5903876.0,16.0,1:22.479,191.219,82.479,Hungaroring,Budapest,Hungary,Lewis Hamilton,McLaren,2009,82.968333,190.096611


In [45]:
circuit_team_winner = results_by_circuit.groupby(['Team Name']).size().reset_index(name='Wins')
circuit_team_winner = circuit_team_winner.sort_values(by='Wins', ascending=False)

In [46]:
circuit_team_winner

,Team Name,Wins
16,Ferrari,206
29,McLaren,145
31,Mercedes,125
34,Red Bull,122
43,Williams,84
35,Renault,30
5,Benetton,22
38,Team Lotus,22
24,Lotus-Climax,14
4,BRM,13
